In [3]:
# ==========================================
# 阿里云天池 - 二手车交易价格预测 (提分版)
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import warnings

# 忽略烦人的警告信息
warnings.filterwarnings('ignore')

# -----------------------------------------------------------
# 1. 配置路径 (请确保这里是你存放csv文件的文件夹路径)
# -----------------------------------------------------------
# 注意：路径前面加 r 是为了防止转义字符报错
BASE_PATH = r'C:\Users\liu\OneDrive\Desktop\used-car-price-predictiondata' 
# 如果你的文件就在当前目录下，可以改成 BASE_PATH = './'

print('Step 1: 正在读取数据...')
# 使用空格作为分隔符读取数据
Train_data = pd.read_csv(f'{BASE_PATH}\\used_car_train_20200313.csv', sep=' ')
TestB_data = pd.read_csv(f'{BASE_PATH}\\used_car_testB_20200421.csv', sep=' ')


# 1. 定义一个处理函数：把 '-' 变成 -1，其他的转成 float
def replace_not_repaired(val):
    if val == '-':
        return -1.0
    else:
        return float(val)

# 2. 应用到 Train 和 Test 数据集上
print('正在修复 notRepairedDamage 列...')
Train_data['notRepairedDamage'] = Train_data['notRepairedDamage'].apply(replace_not_repaired)
TestB_data['notRepairedDamage'] = TestB_data['notRepairedDamage'].apply(replace_not_repaired)

print('修复完成！现在它是数字了。')

print(f'训练集大小: {Train_data.shape}')
print(f'测试集大小: {TestB_data.shape}')

# -----------------------------------------------------------
# 2. 数据清洗与特征工程 (提分关键点!)
# -----------------------------------------------------------
print('Step 2: 正在进行特征工程...')

# 处理异常值：把 train 数据集里 price <= 10 的异常数据删掉
# 这里的逻辑是：几块钱的车可能是废铁或者数据填错了，会干扰模型
Train_data = Train_data[Train_data['price'] > 10]

# --- 关键操作 A：对目标值(价格)进行 Log 变换 ---
# 价格数据通常是长尾分布，取对数后更接近正态分布，模型更容易学习
# 训练时用 log 后的价格，预测出来后再用 exp 变回去
Train_data['price_log'] = np.log1p(Train_data['price'])

# 合并数据以便统一处理特征
# 在 TestB 数据集里加一个临时的 price 列，方便合并
TestB_data['price'] = 0
data = pd.concat([Train_data, TestB_data], ignore_index=True)

# --- 关键操作 B：构造“车龄”特征 ---
# 数据里的日期格式是 20160404 这种数字，要转成时间格式
def date_proc(x):
    m = int(x[4:6])
    if m == 0: m = 1 # 修正月份为0的错误数据
    return x[:4] + '-' + str(m) + '-' + x[6:]

# 对注册日期和上线日期进行转换
data['regDate'] = pd.to_datetime(data['regDate'].astype('str').apply(date_proc))
data['creatDate'] = pd.to_datetime(data['creatDate'].astype('str').apply(date_proc))

# 计算车龄（天数）
data['used_time'] = (data['creatDate'] - data['regDate']).dt.days

# 处理可能出现的负数车龄（数据错误）
data['used_time'] = data['used_time'].apply(lambda x: 0 if x < 0 else x)

print('已新增特征: used_time (车龄)')

# 填补缺失值 (-1 是树模型比较能接受的缺失值标记)
data = data.fillna(-1)

# 删除不需要训练的列
# SaleID: 只是ID，没用
# name: 车型名称太杂，暂时去掉
# regDate, creatDate: 已经提取了 used_time，原日期可以扔了
# price, price_log: 这是我们要预测的答案，不能作为特征
drop_cols = ['SaleID', 'name', 'regDate', 'creatDate', 'price', 'price_log']
feature_cols = [c for c in data.columns if c not in drop_cols]

print(f'使用特征列表: {feature_cols}')

# -----------------------------------------------------------
# 3. 准备训练数据
# -----------------------------------------------------------
print('Step 3: 准备模型输入...')

# 把合并的数据重新拆开
train_df = data[data['price'] > 0].reset_index(drop=True)
test_df = data[data['price'] == 0].reset_index(drop=True)

# 提取特征(X)和标签(Y)
X_data = train_df[feature_cols]
Y_data = train_df['price_log'] # 注意：这里我们要学习的是 Log 后的价格！
X_test = test_df[feature_cols]

# 切分训练集和验证集 (70% 训练，30% 验证)
X_train, X_val, y_train, y_val = train_test_split(X_data, Y_data, test_size=0.3, random_state=2025)

# -----------------------------------------------------------
# 4. 模型训练 (双模型融合)
# -----------------------------------------------------------
print('Step 4: 开始训练 LightGBM...')

# LGBM 参数 (稍微调优了一下)
lgb_model = lgb.LGBMRegressor(
    num_leaves=127,
    n_estimators=1000,      # 树的数量增加到1000
    learning_rate=0.05,     # 学习率
    objective='regression',
    random_state=2025
)
lgb_model.fit(X_train, y_train)

# 验证集评分 (注意：预测出来的是 log 值，要还原回去才能算真实的 MAE)
val_lgb = lgb_model.predict(X_val)
val_lgb_real = np.expm1(val_lgb) # 还原价格
y_val_real = np.expm1(y_val)     # 还原真实价格
mae_lgb = mean_absolute_error(y_val_real, val_lgb_real)
print(f'LightGBM 验证集 MAE: {mae_lgb:.4f}')

# -------------------------------------

print('Step 5: 开始训练 XGBoost...')
xgb_model = xgb.XGBRegressor(
    n_estimators=800,       # 树的数量
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=2025
)
xgb_model.fit(X_train, y_train)

# 验证集评分
val_xgb = xgb_model.predict(X_val)
val_xgb_real = np.expm1(val_xgb) # 还原
mae_xgb = mean_absolute_error(y_val_real, val_xgb_real)
print(f'XGBoost 验证集 MAE: {mae_xgb:.4f}')

# -----------------------------------------------------------
# 5. 模型预测与融合
# -----------------------------------------------------------
print('Step 6: 生成最终预测结果...')

# 预测测试集 (得到的是 Log 后的价格)
pred_lgb_log = lgb_model.predict(X_test)
pred_xgb_log = xgb_model.predict(X_test)

# 还原成真实价格
pred_lgb = np.expm1(pred_lgb_log)
pred_xgb = np.expm1(pred_xgb_log)

# 加权融合 (根据验证集分数分配权重，分数越低权重越高)
# 这里简单一点，给表现好的 LGB 0.6，XGB 0.4
final_pred = pred_lgb * 0.6 + pred_xgb * 0.4

# 修正负值 (虽然 log 变换后很难出现负数，但防一手)
final_pred[final_pred < 0] = 0

# -----------------------------------------------------------
# 6. 生成提交文件
# -----------------------------------------------------------
sub = pd.DataFrame()
sub['SaleID'] = TestB_data['SaleID']
sub['price'] = final_pred
sub.to_csv('./submission_v3_log_feature.csv', index=False)

print('-' * 30)
print('恭喜！运行完成！')
print('生成文件: submission_v3_log_feature.csv')
print(f'预计线上分数将大幅提升 (本地验证 MAE: {mae_lgb * 0.6 + mae_xgb * 0.4:.2f})')

Step 1: 正在读取数据...
正在修复 notRepairedDamage 列...
修复完成！现在它是数字了。
训练集大小: (150000, 31)
测试集大小: (50000, 30)
Step 2: 正在进行特征工程...
已新增特征: used_time (车龄)
使用特征列表: ['model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode', 'seller', 'offerType', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'used_time']
Step 3: 准备模型输入...
Step 4: 开始训练 LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008326 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4910
[LightGBM] [Info] Number of data points in the train set: 105000, number of used features: 25
[LightGBM] [Info] Start training from score 8.036794
LightGBM 验证集 MAE: 537.1953
Step 5: 开始训练 XGBoost...
XGBoost 验证集 MAE: 548.1991
Step 6: 生成最终预测结果...
------------------------------
恭喜！运行完成！
生成文件: submission_v3_log_feature.csv
预计线上分数将大幅提升 (本地验证 MAE: 541.60)
